In [ ]:
import torch
import numpy as np
import torchvision.transforms as transforms
import torchvision
import torch.utils.data as td
import matplotlib.pyplot as plt
import torch.nn as nn
from PIL import Image
import torch.nn.functional as F
import gc
import torchvision.datasets as datasets


test_path = '/content/Testing'
train_path = '/content/Training'

preprocess = transforms.Compose([
      transforms.Resize(256),
      transforms.RandomHorizontalFlip(),
      transforms.RandomVerticalFlip(),
      transforms.RandomAutocontrast(),
      transforms.CenterCrop(224),
      transforms.ToTensor(),
      transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
  ])




train_dataset = datasets.GTSRB(
        root='/content', split='train',
        download=True, transform=preprocess,
    )

test_dataset = datasets.GTSRB(
        root='/content', split='test',
        download=True, transform=preprocess,
    )


train_dataLoader = torch.utils.data.DataLoader(train_dataset, batch_size=256, shuffle=True,  num_workers=2)
print('Train',len(train_dataset))

print('TEST',len(test_dataset))
test_dataLoader = torch.utils.data.DataLoader(test_dataset, batch_size=256, shuffle=True,  num_workers=2)

  0%|          | 0/187490228 [00:00<?, ?it/s]

Extracting /content/gtsrb/GTSRB-Training_fixed.zip to /content/gtsrb


  0%|          | 0/88978620 [00:00<?, ?it/s]

Extracting /content/gtsrb/GTSRB_Final_Test_Images.zip to /content/gtsrb


  0%|          | 0/99620 [00:00<?, ?it/s]

Extracting /content/gtsrb/GTSRB_Final_Test_GT.zip to /content/gtsrb
Train 26640
TEST 12630


In [ ]:
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride = 1, downsample = None):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Sequential(
                        nn.Conv2d(in_channels, out_channels, kernel_size = 3, stride = stride, padding = 1),
                        nn.BatchNorm2d(out_channels),
                        nn.ReLU())
        self.conv2 = nn.Sequential(
                        nn.Conv2d(out_channels, out_channels, kernel_size = 3, stride = 1, padding = 1),
                        nn.BatchNorm2d(out_channels))
        self.downsample = downsample
        self.relu = nn.ReLU()
        self.out_channels = out_channels
        
    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.conv2(out)
        if self.downsample:
            residual = self.downsample(x)
        out += residual
        out = self.relu(out)
        return out
    
class ResNet(nn.Module):
    def __init__(self, block, layers, num_classes = 43):
        super(ResNet, self).__init__()
        self.inplanes = 64
        self.conv1 = nn.Sequential(
                        nn.Conv2d(3, 64, kernel_size = 7, stride = 2, padding = 3),
                        nn.BatchNorm2d(64),
                        nn.ReLU())
        self.maxpool = nn.MaxPool2d(kernel_size = 3, stride = 2, padding = 1)
        self.layer0 = self._make_layer(block, 64, layers[0], stride = 1)
        self.layer1 = self._make_layer(block, 128, layers[1], stride = 2)
        self.layer2 = self._make_layer(block, 256, layers[2], stride = 2)
        self.layer3 = self._make_layer(block, 512, layers[3], stride = 2)
        self.avgpool = nn.AvgPool2d(7, stride=1)
        self.fc = nn.Linear(512, num_classes)
        
    def _make_layer(self, block, planes, blocks, stride=1):
        downsample = None
        if stride != 1 or self.inplanes != planes:
            
            downsample = nn.Sequential(
                nn.Conv2d(self.inplanes, planes, kernel_size=1, stride=stride),
                nn.BatchNorm2d(planes),
            )
        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))
        self.inplanes = planes
        for i in range(1, blocks):
            layers.append(block(self.inplanes, planes))

        return nn.Sequential(*layers)
    
    
    def forward(self, x):
        x = self.conv1(x)
        x = self.maxpool(x)
        x = self.layer0(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)

        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)

        return x

    num_classes = 43
num_epochs = 20
batch_size = 16
learning_rate = 0.01
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

model = ResNet(ResidualBlock, [2, 2, 2, 2]).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
#optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay = 0.001, momentum = 0.9)  
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

# Train the model
total_step = len(train_dataLoader)
training_accuracies = []
training_losses = []
test_accuracies = []
test_losses = []

In [ ]:
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_dataLoader):  
        # Move tensors to the configured device
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        #accuracy
    total = labels.size(0)
    _,predicted = torch.max(outputs.data, 1)
    correct = (predicted == labels).sum().item()
    accuracy = (correct / total) * 100

    del images, labels, outputs
    torch.cuda.empty_cache()
    gc.collect()
    accuracy = (correct / total) * 100
    training_accuracies.append(accuracy)
    training_losses.append(loss.item())

    print ('Training set: Epoch [{}/{}], Loss: {:.4f},  Accuracy: {:.2f}%, ' 
                .format(epoch+1, num_epochs, loss.item(), accuracy))

    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in test_dataLoader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            loss_test = criterion(outputs, labels)
            del images, labels, outputs
        accuracy = (correct / total) * 100    
        test_accuracies.append(accuracy)
        test_losses.append(loss_test.item())
        print ('Test set: Epoch [{}/{}], Loss: {:.4f},  Accuracy: {:.2f}%, ' 
                    .format(epoch+1, num_epochs, loss_test.item(), accuracy))

Training set: Epoch [1/20], Loss: 2.8623,  Accuracy: 18.75%, 
Test set: Epoch [1/20], Loss: 3.0331,  Accuracy: 22.27%, 
Training set: Epoch [2/20], Loss: 2.0186,  Accuracy: 43.75%, 
Test set: Epoch [2/20], Loss: 2.6657,  Accuracy: 26.14%, 
Training set: Epoch [3/20], Loss: 1.5344,  Accuracy: 68.75%, 
Test set: Epoch [3/20], Loss: 2.3871,  Accuracy: 31.43%, 
Training set: Epoch [4/20], Loss: 0.9037,  Accuracy: 81.25%, 
Test set: Epoch [4/20], Loss: 3.0469,  Accuracy: 28.32%, 
Training set: Epoch [5/20], Loss: 0.7836,  Accuracy: 81.25%, 
Test set: Epoch [5/20], Loss: 1.8459,  Accuracy: 45.91%, 
Training set: Epoch [6/20], Loss: 0.8919,  Accuracy: 81.25%, 
Test set: Epoch [6/20], Loss: 2.5312,  Accuracy: 34.21%, 
Training set: Epoch [7/20], Loss: 0.7385,  Accuracy: 75.00%, 
Test set: Epoch [7/20], Loss: 2.5860,  Accuracy: 47.55%, 
Training set: Epoch [8/20], Loss: 0.5067,  Accuracy: 93.75%, 
Test set: Epoch [8/20], Loss: 1.8455,  Accuracy: 55.65%, 
Training set: Epoch [9/20], Loss: 0.1439

In [ ]:
print(f"training losses:{training_losses}")
print(f"training accuracies:{training_accuracies}")
print(f"test losses:{test_losses}")
print(f"test accuracies:{test_accuracies}")

training losses:[2.8622536659240723, 2.0185539722442627, 1.5343759059906006, 0.903653085231781, 0.7836331129074097, 0.8919228315353394, 0.7385133504867554, 0.5066928863525391, 0.14391857385635376, 0.6484894752502441, 0.3750837445259094, 0.04017014801502228, 0.053187012672424316, 0.19116321206092834, 0.04156258702278137, 0.1450934261083603, 0.048421841114759445, 0.04420997574925423, 0.07083585858345032, 0.04736204072833061]
training accuracies:[18.75, 43.75, 68.75, 81.25, 81.25, 81.25, 75.0, 93.75, 100.0, 81.25, 93.75, 100.0, 100.0, 100.0, 100.0, 93.75, 100.0, 100.0, 100.0, 100.0]
test losses:[3.0331342220306396, 2.6657207012176514, 2.3871335983276367, 3.046919822692871, 1.8458881378173828, 2.531233549118042, 2.5859732627868652, 1.8455193042755127, 1.2813313007354736, 2.3492510318756104, 1.412337064743042, 0.511991024017334, 0.60981684923172, 0.6591342091560364, 0.42896831035614014, 0.6913515329360962, 0.7747753262519836, 0.6989102363586426, 0.6023792624473572, 0.7633125185966492]
test 